In [1]:
import re
import random
import sys, os
sys.path.append(os.path.abspath("./"))
from utils import *

sys.path.append(os.path.abspath("../"))
import voc_annotation

## Strategy
1. Keep every classified car as is
2. Convert *bus*, *aeroplane*, *motorbike*, *train*, *refrigerator*, *boat*, *toilet*, *truck*, *parkingmeter*, *suitcase* to **car**
3. Remove non-classified images (aka images with no bounding boxes)

In [2]:
output_folder        = '../data/output/mhca/'
classes_path   = '../model_data/coco_classes.txt'
detection_file_path = output_folder + 'detection_output.txt'

trainval_output_file = output_folder + 'trainval_annotations.txt'
test_output_file     = output_folder + 'test_annotations.txt'
output_file          = output_folder + 'prepped_detection_output.txt'
non_class_file       = output_folder + 'non_classified.txt'

classes  = get_classes(classes_path)
id2class = {str(i): classes[i] for i in range(len(classes))}

img2bbs  = get_predictions(detection_file_path)
img2bb   = explode_dict(img2bbs)

original_img2bbs = img2bbs.copy()
original_img2bb  = img2bb.copy()

non_classified_image_paths = [path for path, bbs in img2bbs.items() if len(bbs) == 0]

- leave 30% for test
- take other classes

# MHCA

## Filter Bounding Boxes

### Change labels to car
- **input**:
- **output**:

In [3]:
labels_gonna_be_car = set(['7', '5', '12', '3', '28', '6', '72', '4', '61', '8'])
for i in range(0, len(img2bb)):
    label = img2bb[i][1].prediction
    if label in labels_gonna_be_car:
        img2bb[i][1].prediction = '2'

### Filter non car Boounding Boxes
- **input**:
- **output**:

In [4]:
img2bb = [elem for elem in img2bb if elem[1].prediction == '2']

## Results

In [5]:
img2bbs = back_to_dict(img2bb)
print(f'NUMBER OF BOXES BEFORE:   {len(original_img2bb)}')
print(f'NUMBER OF BOXES AFTER:    {len(img2bb)}\n')

print(f'NUMBER OF IMAGES BEFORE:  {len(original_img2bbs)}')
print(f'NUMBER OF IMAGES AFTER:   {len(img2bbs)}')

NUMBER OF BOXES BEFORE:   20203
NUMBER OF BOXES AFTER:    19512

NUMBER OF IMAGES BEFORE:  16481
NUMBER OF IMAGES AFTER:   14378


## Split Train and Test for MHCA
- **input**:
- **output**:

In [6]:
random.seed(42)
paths = list(img2bbs.keys())
random.shuffle(paths)

train_percentage = 0.8
test_percentage  = 1 - train_percentage

train_paths = random.sample(paths, int(len(paths) * train_percentage))
test_paths  = list(set(paths) - set(train_paths))

train_mhca_img2bbs = {path: img2bbs[path] for path in train_paths}
test_mhca_img2bbs  = {path: img2bbs[path] for path in test_paths}

print(f'NUMBER OF TRAIN INSTANCES FOR MHCA: {len(train_mhca_img2bbs)}')
print(f'NUMBER OF TEST  INSTANCES FOR MHCA: {len(test_mhca_img2bbs)}')

NUMBER OF TRAIN INSTANCES FOR MHCA: 11502
NUMBER OF TEST  INSTANCES FOR MHCA: 2876


# VOC

## Convert VOC format to YOLO format
- **input**: VOC annotation file
- **output**: converted file with YOLO style annotations

In [7]:
voc_path    = Path('../data/input/voc/voc2012/VOCdevkit-trainval/')
output_path = Path('../data/output/voc/voc2012/')
output_path.mkdir(parents=True, exist_ok=True)

voc_annotation.convert(voc_folder=str(voc_path), output_path=str(output_path))

File Not Found for test: No such file or directory
All good


## Change VOC labels to COCO labels
- **input**: file with YOLO style annotations
- **output**: same file but with COCO classes

In [8]:
converted_voc_train_annotations_path = Path(output_path, 'converted_2012_train.txt')
converted_voc_val_annotations_path   = Path(output_path, 'converted_2012_val.txt')
regex = r'\b(\w+)$' # regex to find last word

# class mappings
voc2coco = {
    '0': '5',
    '1': '1',
    '2': '14',
    '3': '8',
    '4': '39',
    '5': '5',
    '6': '2',
    '7': '15',
    '8': '56',
    '9': '19',
    '10': '60',
    '11': '16',
    '12': '17',
    '13': '3',
    '14': '0',
    '15': '58',
    '16': '19',
    '17': '57',
    '18': '6',
    '19': '62'
}

##### Run this only ONE time **for pizza sake** !

In [9]:
replace_in_file(str(converted_voc_train_annotations_path), regex, voc2coco)
replace_in_file(str(converted_voc_val_annotations_path), regex, voc2coco)

All good
All good


# Combine MHCA with VOC
- **input**:
- **output**:

## Get VOC sets

In [10]:
img2bbs_train = get_predictions(converted_voc_train_annotations_path)
img2bb_train  = explode_dict(img2bbs_train)
img2bbs_val   = get_predictions(converted_voc_val_annotations_path)
img2bb_val    = explode_dict(img2bbs_val)

voc_img2bbs = {**img2bbs_train, **img2bbs_val}
voc_img2bb  = img2bb_val + img2bb_train

print(f'NUMBER OF TRAIN INSTANCES FOR VOC: {len(img2bbs_train)}')
print(f'NUMBER OF TEST  INSTANCES FOR VOC: {len(img2bbs_val)}\n')
print(f'NUMBER OF TOTAL INSTANCES FOR VOC: {len(voc_img2bbs)}')

NUMBER OF TRAIN INSTANCES FOR VOC: 5717
NUMBER OF TEST  INSTANCES FOR VOC: 5823

NUMBER OF TOTAL INSTANCES FOR VOC: 11540


## Merge VOC and MHCA

In [12]:
total_img2bbs_trainval = train_mhca_img2bbs.copy()
total_img2bbs_test     = {**voc_img2bbs, **test_mhca_img2bbs}

print(f'NUMBER OF TRAIN TOTAL INSTANCES: {len(total_img2bbs_trainval)}')
print(f'NUMBER OF TEST  TOTAL INSTANCES: {len(total_img2bbs_test)}')

NUMBER OF TRAIN TOTAL INSTANCES: 11502
NUMBER OF TEST  TOTAL INSTANCES: 14416


## Write to file

In [13]:
write_annotations(total_img2bbs_trainval, trainval_output_file)
write_annotations(total_img2bbs_test, test_output_file)

All good
All good


In [14]:
write_list_to_file(non_classified_image_paths, non_class_file)

All good
